<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

## Прогнозирование стоимости автомобиля по характеристикам
*Этот Ноутбук является Примером/Шаблоном (Baseline) к этому соревнованию и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.


> **baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline является хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder

from sklearn.feature_selection import f_classif, mutual_info_classif
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import (AdaBoostClassifier, GradientBoostingClassifier,
                              RandomForestClassifier, ExtraTreesClassifier)

In [ ]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# Setup

In [ ]:
VERSION    = 1
DIR_TRAIN  = '../input/parsing-all-moscow-auto-ru-09-09-2020/' # подключил к ноутбуку внешний датасет
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%
#VAL_SIZE   = 0.33   # 33%
N_FOLDS    = 5
#CATBOOST
ITERATIONS = 1000
LR         = 0.1

# Data

In [ ]:
!ls '../input'

In [ ]:
train = pd.read_csv(DIR_TRAIN+'all_auto_ru_09_09_2020.csv') # датасет для обучения модели
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

In [ ]:
sample_submission.info()

In [ ]:
train.info()

In [ ]:
test.info()

## Data Preprocessing

Унифицируем названия элементов в признаках в test и train (в коде не буду оставлять просмотр каждого элемента, так как будет объемно и сложно в поиске)

# 1.в test в признаке model_name записано тоже самое, что и в model в train. Переименуем

In [ ]:
test.rename(columns=lambda x: x.replace('model_name', 'model'), inplace=True)

In [ ]:
train.dropna(subset=['price'], inplace=True)

# Парсим новые данные.

Новые данные с глубиной 60 дней
На kaggle не работает. Скидываю закомментированный код, которым получал новые данные

HEADERS = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
    'Connection': 'keep-alive',
    'Content-Length': '137',
    'content-type': 'application/json',
    'Cookie': '_csrf_token=1c0ed592ec162073ac34d79ce511f0e50d195f763abd8c24; autoru_sid=a%3Ag5e3b198b299o5jhpv6nlk0ro4daqbpf.fa3630dbc880ea80147c661111fb3270%7C1580931467355.604800.8HnYnADZ6dSuzP1gctE0Fw.cd59AHgDSjoJxSYHCHfDUoj-f2orbR5pKj6U0ddu1G4; autoruuid=g5e3b198b299o5jhpv6nlk0ro4daqbpf.fa3630dbc880ea80147c661111fb3270; suid=48a075680eac323f3f9ad5304157467a.bc50c5bde34519f174ccdba0bd791787; from_lifetime=1580933172327; from=yandex; X-Vertis-DC=myt; crookie=bp+bI7U7P7sm6q0mpUwAgWZrbzx3jePMKp8OPHqMwu9FdPseXCTs3bUqyAjp1fRRTDJ9Z5RZEdQLKToDLIpc7dWxb90=; cmtchd=MTU4MDkzMTQ3MjU0NQ==; yandexuid=1758388111580931457; bltsr=1; ',
    'Host': 'auto.ru',
    'origin': 'https://auto.ru/cars/all/',
    'Referer': 'https://auto.ru/cars/all/',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0',
    'x-client-app-version': '202002.03.092255',
    'x-client-date': '1580933207763',
    'x-csrf-token': '1c0ed592ec162073ac34d79ce511f0e50d195f763abd8c24',
    'x-page-request-id': '60142cd4f0c0edf51f96fd0134c6f02a',
    'x-requested-with': 'fetch'
}

dict_columns = {'bodyType':0, 'brand':0, 'color':0, 'fuelType':0, 'modelDate':0, 
                'name':0, 'numberOfDoors':0, 'productionDate':0, 
                'vehicleConfiguration':0, 'vehicleTransmission':0, 
                'engineDisplacement':0, 'enginePower':0, 'description':0, 
                'mileage':0, 'Комплектация':0, 'Привод':0, 'Руль':0, 
                'Состояние':0, 'Владельцы':0, 'ПТС':0, 'Таможня':0, 
                'Владение':0, 'price':0, 'model':0}

#model = 'M6'  
создаем функцию для обработки 23 необходимых столбцов
def from_elem_to_dict(d_elem, d_dict):
    utils_clean_values_dict(d_dict)
    
    try: d_dict['bodyType'] = d_elem['vehicle_info']['configuration']['human_name']
    except: d_dict['bodyType'] = None

    try: d_dict['brand'] = d_elem['vehicle_info']['mark_info']['code']
    except: d_dict['brand'] = None

    try: d_dict['color'] = d_elem['color_hex']
    except: d_dict['color'] = None

    try: d_dict['fuelType'] = d_elem['lk_summary'].split()[-1]
    except: d_dict['fuelType'] = None

    try: d_dict['modelDate'] = d_elem['vehicle_info']['super_gen']['year_from']
    except: d_dict['modelDate'] = None

    try: d_dict['name'] =  d_elem['vehicle_info']['tech_param']['human_name']
    except: d_dict['name'] = None

    try: d_dict['numberOfDoors'] = d_elem['vehicle_info']['configuration']['doors_count']
    except: d_dict['numberOfDoors'] = None

    try: d_dict['productionDate'] = d_elem['documents']['year']
    except: d_dict['productionDate'] = None

    try: d_dict['vehicleTransmission'] = d_elem['vehicle_info']['tech_param']['transmission']
    except: d_dict['vehicleTransmission'] = None

    try: d_dict['engineDisplacement'] = d_elem['vehicle_info']['tech_param']['human_name'].split()[0]
    except: d_dict['engineDisplacement'] = None

    try: d_dict['vehicleConfiguration'] = elem['vehicle_info']['configuration']['body_type']+' '+d_dict['vehicleTransmission']+' '+d_dict['engineDisplacement']
    except: d_dict['vehicleConfiguration'] = None

    try: d_dict['enginePower'] = d_elem['vehicle_info']['tech_param']['power']
    except: d_dict['enginePower'] = None

    try: d_dict['description'] = d_elem['description']
    except: d_dict['description'] = None

    try: d_dict['mileage'] = d_elem['state']['mileage']
    except: d_dict['mileage'] = None

    try: d_dict['Комплектация'] = d_elem['vehicle_info']['complectation']
    except: d_dict['Комплектация'] = None

    try: d_dict['Привод'] = d_elem['lk_summary'].split(', ')[-2]
    except: d_dict['Привод'] = None

    try: d_dict['Руль'] = d_elem['vehicle_info']['steering_wheel']
    except: d_dict['Руль'] = None

    # 'Состояние' не найдено
    d_dict['Состояние'] = None

    try: d_dict['Владельцы'] = d_elem['documents']['owners_number']
    except: d_dict['Владельцы'] = None

    try: d_dict['ПТС'] = d_elem['documents']['pts']
    except: d_dict['ПТС'] = None

    try: d_dict['Таможня'] = d_elem['documents']['custom_cleared']
    except: d_dict['Таможня'] = None

    try: d_dict['Владение'] =  d_elem['documents']['purchase_date']
    except: d_dict['Владение'] = None


    try: d_dict['price'] = d_elem['price_info']['RUR']
    except: d_dict['price'] = None

    # дата размещения объявления решил добавить 
    try: d_dict['start_date'] =  d_elem['additional_info']['hot_info']['start_time']
    except: d_dict['start_date'] = None

    # статус объявления 
    try: d_dict['hidden'] =  d_elem['additional_info']['hidden']
    except: d_dict['hidden'] = None

    # модель 
    #try:d_dict['model'] =  d_elem['model']
    #except: d_dict['model'] = None
        
    try: d_dict['model'] = d_elem['vehicle_info']['model_info']['code']
    except: d_dict['model'] = None

    return

from_elem_to_dict(elem, dict_columns)

auto_ru = {}

iter_of_ads, iter_of_marks, old_iter_of_ads, sum_ads, sum_ads_from_dict = 0,0,0,0,0
space = ' '
    
for num_page in range(1, 300):
    param = {
                'section': "all",
                'category': "cars",
                'sort': "fresh_relevance_1-desc",
                "top_days":"60",
                "geo_radius":200,
                "geo_id":[213],
                'page': num_page
    }
    response = requests.post(url, json=param, headers = HEADERS)
    data = response.json()

    if data['offers'] != []:
        for elem in data['offers']:
            from_elem_to_dict(elem, dict_columns)
            auto_ru[iter_of_ads] = dict_columns.copy()
            iter_of_ads += 1
    else:
        break
iter_of_marks +=1

df_auto_ru = pd.DataFrame(auto_ru).T
df_auto_ru.to_csv('D:\\auto2.csv', index=False)

print(f'кол-во строк датасета:= {len(df_auto_ru)}')

подгружаем

In [ ]:
DIR_TRAIN2  = '../input/auto-13122020/'
train_new = pd.read_csv(DIR_TRAIN2+'auto 13.12.2020.csv') 

In [ ]:
train_new.info()

In [ ]:
train_new.dropna(subset=['price'], inplace=True)

Объединяем трейн и тест в один датасет и оставляем только пересекающиеся признаки

In [ ]:
columns=['brand','bodyType','color','fuelType', 'modelDate','numberOfDoors','productionDate',
         'vehicleConfiguration','engineDisplacement','enginePower','mileage','Привод','Руль','Владельцы','ПТС','price','model']
columns2=['brand','bodyType','color','fuelType', 'modelDate','numberOfDoors','productionDate',
         'vehicleConfiguration','engineDisplacement','enginePower','mileage','Привод','Руль','Владельцы','ПТС','model']
df_train = train[columns]
df_test = test[columns2]
df_train_new = train_new[columns]

In [ ]:
# удалим строки, где отсутствует цена
df_train = df_train.dropna(axis=0, subset=['price'])
df_train_new = df_train_new.dropna(axis=0, subset=['price'])

In [ ]:
df_train['sample'] = 1 # помечаем где у нас трейн
df_train_new['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест

data = df_test.append(df_train, sort=False).reset_index(drop=True)

In [ ]:
data = data.append(df_train_new, sort=False).reset_index(drop=True)

In [ ]:
data.info()

# Пройдёмся по признакам

# 1.bodyType

Упростим названия кузовов

In [ ]:
data.bodyType.value_counts()

In [ ]:
data.bodyType = data.bodyType.astype(str).apply(lambda x: None if x.strip()=='' else x)
data.bodyType = data.bodyType.apply(lambda x: x.split(' ')[0].lower())
#train['bodyType'] = train['bodyType'].apply(lambda x: x.lower())

In [ ]:
data.bodyType.value_counts()

In [ ]:
data.bodyType.value_counts().plot.barh()

пустые значения заменим самым популярным

In [ ]:
data.bodyType = data.bodyType.replace('nan', 'внедорожник')

# 2.Color

In [ ]:
data.color.value_counts()

Приведем к единообразию написания

In [ ]:
color_dict = {'CACECB': 'серебристый', 
              'FAFBFB':'белый', 
              'EE1D19':'красный', 
              '97948F':'серый',
              '660099':'пурпурный',
              '040001':'чёрный',
              '4A2197':'фиолетовый',
              '200204':'коричневый',
              '0000CC':'синий',
              '007F00':'зелёный',
              'C49648':'бежевый',
              '22A0F8':'голубой',
              'DEA522':'золотистый',
              'FFD600': 'жёлтый',
              'FF8649':'оранжевый',
              'FFC0CB':'розовый',
              'серебристый': 'серебристый', 
              'белый':'белый', 
              'красный':'красный', 
              'серый':'серый',
              'пурпурный':'пурпурный',
              'чёрный':'чёрный',
              'фиолетовый':'фиолетовый',
              'коричневый':'коричневый',
              'синий':'синий',
              'зелёный':'зелёный',
              'бежевый':'бежевый',
              'голубой':'голубой',
              'золотистый':'золотистый',
              'жёлтый': 'жёлтый',
              'оранжевый':'оранжевый',
              'розовый':'розовый'
             }
data.color = data.color.map(color_dict)

In [ ]:
data.color.value_counts()

In [ ]:
data.color = data.color.fillna('черный')

# 3.modelDate и productionDate

In [ ]:
data.modelDate.value_counts()

Заменим пустые значения на самые популярные

In [ ]:
data.modelDate = data.modelDate.fillna('2010')
data.modelDate = data.modelDate.astype('int')

In [ ]:
data.modelDate = 2020 - data.modelDate # возраст модели
data.productionDate = 2020 - data.productionDate # возраст машины

# 4.numberOfDoors

In [ ]:
data.numberOfDoors.value_counts()

In [ ]:
data.numberOfDoors = data.numberOfDoors.fillna('5.0')

# 5.vehicleConfiguration

In [ ]:
data.vehicleConfiguration.value_counts()

In [ ]:
data.vehicleConfiguration = data.vehicleConfiguration.fillna('SEDAN MECHANICAL 1.6')

In [ ]:
data.vehicleConfiguration = data.vehicleConfiguration.astype(str).apply(lambda x: None if x.strip()=='' else x)
data.vehicleConfiguration = data.vehicleConfiguration.apply(lambda x: x.split()[1])

In [ ]:
data.vehicleConfiguration.value_counts()

# 6.engineDisplacement

In [ ]:
data.engineDisplacement.value_counts()

In [ ]:
data.engineDisplacement = data.engineDisplacement.astype(str).apply(lambda x: None if x.strip()=='' else x)
data.engineDisplacement = data.engineDisplacement.apply(lambda x: x[:3])

есть ряд кривых значений. Заменим кривые на популярные

In [ ]:
engine_Dis = {'1.0': '1.0', 
              '1.1':'1.1', 
              '1.2':'1.2', 
              '1.3':'1.3',
              '1.4':'1.4',
              '1.5':'1.5',
              '1.6':'1.6',
              '1.7':'1.7',
              '1.8':'1.8',
              '1.9':'1.9',
              '2.0':'2.0',
              '2.1':'2.1',
              '2.2':'2.2',
              '2.3':'2.3',
              '2.4':'2.4',
              '2.5':'2.5',
              '2.6':'2.6',
              '2.7':'2.7',
              '2.8':'2.8',
              '2.9':'2.9',
              '3.0':'3.0',
              '3.1':'3.1',
              '3.2':'3.2',
              '3.3':'3.3',
              '3.4':'3.4',
              '3.5':'3.5',
              '3.6':'3.6',
              '3.7':'3.7',
              '3.8':'3.8',
              '3.9':'3.9',
              '4.0':'4.0',
              '4.1':'4.1',
              '4.2':'4.2',
              '4.3':'4.3',
              '4.4':'4.4',
              '4.5':'4.5',
              '4.6':'4.6',
              '4.7':'4.7',
              '4.8':'4.8',
              '4.9':'4.9',
              '5.0':'5.0',
             }
data.engineDisplacement = data.engineDisplacement.map(engine_Dis)

пустые заменим на популырные объемы

In [ ]:
data.engineDisplacement = data.engineDisplacement.fillna('1.6')

# 7.enginePower

In [ ]:
data.enginePower.value_counts()

Разделим на категории по мощности 

In [ ]:
data.enginePower = data.enginePower.astype(str).apply(lambda x: None if x.strip()=='' else x)
data.enginePower = data.enginePower.apply(lambda x: x.split()[0])
data.enginePower = data.enginePower.astype(float)

In [ ]:
def engine_power(x):
    if x < 100: x = 1
    elif 99 < x < 150: x = 2
    elif 149 < x < 200: x = 3
    elif 199 < x < 250: x = 4
    elif 249 < x < 300: x = 5
    elif 299 < x < 350: x = 6
    elif 349 < x < 400: x = 7
    elif 399 < x < 450: x = 8
    elif 449 < x < 500: x = 9
    elif 499 < x < 550: x = 10
    elif 549 < x < 600: x = 11
    elif 599 < x: x = 12
    return x  
data.enginePower = data.enginePower.map(engine_power)

In [ ]:
data.enginePower = data.enginePower.fillna('2.0')

In [ ]:
data.enginePower.value_counts()

# 8.mileage

In [ ]:
data.mileage.describe()

поделим на категории

In [ ]:
def mile_age(x):
    if x < 20000: x = 1
    elif 19999 < x < 40000: x = 2
    elif 39999 < x < 60000: x = 3
    elif 59999 < x < 80000: x = 4
    elif 79999 < x < 100000: x = 5
    elif 99999 < x < 200000: x = 6
    elif 199999 < x < 500000: x = 7
    elif 499999 < x: x = 8
    return x  
data.mileage = data.mileage.map(mile_age)

заменим пустые значения по Привод на передний

In [ ]:
data.Привод = data.Привод.fillna('передний')

# 9.Руль

In [ ]:
data.Руль.value_counts()

In [ ]:
wheel = {'LEFT': 'LEFT', 
              'Левый':'LEFT', 
              'RIGHT':'RIGHT', 
              'Правый':'RIGHT',

             }
data.Руль = data.Руль.map(wheel)
data.Руль = data.Руль.fillna('LEFT')

# 10.Владельцы

In [ ]:
data.Владельцы.value_counts()

In [ ]:
data.Владельцы = data.Владельцы.astype(str).apply(lambda x: None if x.strip()=='' else x)

In [ ]:
owner = {'3.0': '3.0','2.0':'2.0','1.0':'1.0', 
        '3 или более':'3.0',
        '1 владелец':'1.0',
        '2 владельца':'2.0',
         }
data.Владельцы = data.Владельцы.map(owner)
#data.Владельцы = data.Владельцы.fillna('3.0')

In [ ]:
data.Владельцы = data.Владельцы.fillna('3.0')

In [ ]:
data.Владельцы.value_counts()

# 11.ПТС

In [ ]:
data.ПТС.value_counts()

In [ ]:
pts = {'ORIGINAL': 'ORIGINAL', 
              'Оригинал':'ORIGINAL', 
              'DUPLICATE':'DUPLICATE', 
              'Дубликат':'DUPLICATE',

             }
data.ПТС = data.ПТС.map(pts)
data.ПТС = data.ПТС.fillna('ORIGINAL')

In [ ]:
data.ПТС.value_counts()

In [ ]:
data.info()

In [ ]:
# удалим строки, где отсутствует цена
df_train = df_train.dropna(axis=0, subset=['price'])
df_train_new = df_train_new.dropna(axis=0, subset=['price'])

## Label Encoding

In [ ]:
bin_cols = ['Руль','ПТС']
cat_cols = ['brand','bodyType','color','fuelType','numberOfDoors','vehicleConfiguration', 'engineDisplacement',
            'enginePower','mileage','Привод','Владельцы','model']
num_cols = ['modelDate','productionDate']

Переведем каждое значение бинарных признаков в уникальное число при помощи LabelEncoder()

In [ ]:
label_encoder = LabelEncoder()
for column in bin_cols:
    data[column] = label_encoder.fit_transform(data[column])
data.head()

In [ ]:
data[bin_cols].head()

In [ ]:
temp_df = data[data['sample']==1]
temp_df.info()

Переведём на язык программы

In [ ]:
for colum in ['brand','bodyType','color','fuelType','numberOfDoors',
              'vehicleConfiguration', 'engineDisplacement','enginePower','mileage','Привод','Владельцы','model']:
    data[colum] = data[colum].astype('category').cat.codes

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
X = data.query('sample == 1').drop(['sample','price'], axis=1)
X_sub = data.query('sample == 0').drop(['sample','price'], axis=1)

## Train Split

In [ ]:
#y = train['price']
y = temp_df['price']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

# Model 1 : CatBoost

In [ ]:
#model = CatBoostRegressor(iterations = ITERATIONS,
#                          learning_rate = LR,
#                          random_seed = RANDOM_SEED,
#                          eval_metric='MAPE',
#                          custom_metric=['R2', 'MAE']
#                         )
#model.fit(X_train, y_train,
#         #cat_features=cat_features_ids,
#         eval_set=(X_test, y_test),
#         verbose_eval=100,
#         use_best_model=True,
#         plot=True
#         )

In [ ]:
#predict = model.predict(X_test)

# оцениваем точность
#print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

17.95495 на kaggle

# Model 2:GradientBoostingRegressor

In [ ]:
def gb_model (y,x):
    gb = GradientBoostingRegressor(min_samples_split=2, learning_rate=0.03, max_depth=10, n_estimators=1000)
    gb.fit(x,y)
    return gb

def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [ ]:
#gb_model = gb_model(y_train,X_train)

In [ ]:
#test_predict = gb_model.predict(X_test)
#test_score = mape(y_test, test_predict)
#print(f"Точность модели по метрике MAPE: {mape(y_test, test_predict):0.3f}")

14.47140 на kaggle  
с новыми парсенными данными результаты на kaggle 14.88357

# Model 3:ExtraTreesRegressor[](http://)

In [ ]:
def et_model (y,x):
    et = ExtraTreesRegressor(max_depth=15,max_features = 'sqrt',
                             n_estimators = 1000, random_state = 42, n_jobs = -1)
    et.fit(x,y)
    return et

In [ ]:
#et_model = et_model(y_train,X_train)

In [ ]:
#test_predict = et_model.predict(X_test)
#test_score = mape(y_test, test_predict)
#print(f"Точность модели по метрике MAPE: {mape(y_test, test_predict):0.3f}")

21.20325 на kaggle

# Model 4: RandomForestRegressor

In [ ]:
def reg_model(y, x):
    reg = RandomForestRegressor(n_estimators=1000, n_jobs = -1, max_depth = 15, max_features = 'log2', random_state = 42, oob_score=True,) 
    reg.fit(x,y) 
    return reg

In [ ]:
#reg_model = reg_model(y_train,X_train)

In [ ]:
#test_predict = reg_model.predict(X_test)
#test_score = mape(y_test, test_predict)
#print(f"Точность модели по метрике MAPE: {mape(y_test, test_predict):0.3f}")

16.99574 на kaggle

# Подбор параметров модели 

заккоментим, так как отрабатывает нескольок часов

In [ ]:
#from sklearn.model_selection import GridSearchCV

# Добавим типы регуляризации
#max_depth = [5, 10,15]

# Зададим ограничения для параметра регуляризации
#n_estimators = [100, 200,1000]


# Создадим гиперпараметры
#hyperparameters = dict(max_depth=max_depth,n_estimators=n_estimators)

#model = RandomForestRegressor()
#model.fit(X_train, y_train)

#clf = GridSearchCV(model, hyperparameters)

#best_model = clf.fit(X_train, y_train)

# View best hyperparameters
#print('Лучшее max_depth:', best_model.best_estimator_.get_params()['max_depth'])
#print('Лучшее n_estimators:', best_model.best_estimator_.get_params()['n_estimators'])

Лучшее max_depth: 15
Лучшее n_estimators: 1000

In [ ]:
#from sklearn.model_selection import GridSearchCV
#max_depth = [5, 10,15]
#n_estimators = [100, 200,1000]


#Создадим гиперпараметры
#hyperparameters = dict(max_depth=max_depth,n_estimators=n_estimators)

#model =  GradientBoostingRegressor()
#model.fit(X_train, y_train)

#clf = GridSearchCV(model, hyperparameters)

#best_model = clf.fit(X_train, y_train)

 #View best hyperparameters
#print('Лучшее max_depth:', best_model.best_estimator_.get_params()['max_depth'])
#print('Лучшее n_estimators:', best_model.best_estimator_.get_params()['n_estimators'])

# Стекинг

In [ ]:
#def compute_meta_feature(clf, X_train, X_test, y_train, cv):
#    
#    n_classes = len(np.unique(y_train))
#    X_meta_train = np.zeros((len(y_train), n_classes), dtype=np.float32)
#
#    splits = cv.split(X_train)
#    for train_fold_index, predict_fold_index in splits:
#        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
#        y_fold_train = y_train[train_fold_index]
#        
#        folded_clf = clone(clf)
#        folded_clf.fit(X_fold_train, y_fold_train)
#        
#        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)
#    
##    meta_clf = clone(clf)
#    meta_clf.fit(X_train, y_train)
#    
#    X_meta_test = meta_clf.predict_proba(X_test)
#    
#    return X_meta_train, X_meta_test

In [ ]:
#def generate_meta_features(classifiers, X_train, X_test, y_train, cv):
#   
#    features = [
#        compute_meta_feature(clf, X_train, X_test, y_train, cv)
#        for clf in tqdm(classifiers)
#    ]
#    
#    stacked_features_train = np.hstack([
#        features_train for features_train, features_test in features
#    ])
#
#    stacked_features_test = np.hstack([
#        features_test for features_train, features_test in features
#    ])
#    
#    return stacked_features_train, stacked_features_test

In [ ]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)
#cv = KFold(n_splits=N_FOLDS, shuffle=True)

In [ ]:
X_train = np.where(np.isnan(X_train), 0, X_train)
X_test = np.where(np.isnan(X_test), 0, X_test)
y_train = np.where(np.isnan(y_train), 0, y_train)

In [ ]:
from sklearn.base import clone
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    
    #n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros(len(y_train), dtype=np.float32)

    splits = cv.split(X_train)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        
        X_meta_train[predict_fold_index] = folded_clf.predict(X_fold_predict)
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict(X_test)
    
    n = cv.n_splits
    X_meta_test = X_meta_test / n

    
    return X_meta_train, X_meta_test

In [ ]:
def generate_meta_features(classifiers, X_train, X_test, y_train, cv):
   
    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(classifiers)
    ]
    
    stacked_features_train = np.hstack([
        features_train for features_train, features_test in features
    ])

    stacked_features_test = np.hstack([
        features_test for features_train, features_test in features
    ])
    
    return stacked_features_train, stacked_features_test

In [ ]:
#clf = KNeighborsRegressor()
#KNeighborsClassifier()

In [ ]:
#clf = KNeighborsRegressor()
#n_classes = len(np.unique(y_train))
#X_meta_train = np.zeros((len(y_train), n_classes), dtype=np.float32)
#X_meta_train = np.zeros(len(y_train), dtype = np.float32)
#splits = cv.split(X_train)
#for train_fold_index, predict_fold_index in splits:
#    folded_clf = clone(clf)
#    folded_clf.fit(X_train[train_fold_index], y_train[train_fold_index])
#    X_meta_train[predict_fold_index] = folded_clf.predict(X_train[predict_fold_index])
#meta_clf = clone(clf)
#meta_clf.fit(X_train, y_train)
#X_meta_test = meta_clf.predict(X_test)

In [ ]:
stacked_features_train, stacked_features_test = generate_meta_features([
    #RandomForestRegressor(n_estimators=100),
    #GradientBoostingClassifier
    #GradientBoostingRegressor(n_estimators=100),
    #KNeighborsRegressor()
    KNeighborsClassifier()
    #ExtraTreesClassifier(n_estimators=100)
], X_train, X_test, y_train, cv)

In [ ]:
stacked_features_train.shape

In [ ]:
y_train.shape

In [ ]:
stacked_features_train = stacked_features_train.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)

In [ ]:
stacked_features_train.shape

In [ ]:
y_train.shape

In [ ]:
#y_train.values.ravel()

In [ ]:
#total_features_train = np.hstack([X_train, stacked_features_train])
#total_features_test = np.hstack([X_test, stacked_features_test])
#np.random.seed(42)
#clf = ExtraTreesClassifier(n_estimators=100)
#clf = GradientBoostingClassifier()
#GradientBoostingRegressor(n_estimators=100),
#clf = KNeighborsRegressor()
#clf = KNeighborsClassifier()
#clf = LogisticRegression(penalty='none', multi_class='auto', solver='lbfgs')
#clf.fit(stacked_features_train, y_train)

In [ ]:
test_predict = clf.predict(X_test)
test_score = mape(y_test, test_predict)
print(f"Точность модели по метрике MAPE: {mape(y_test, test_predict):0.3f}")

# Submission

In [ ]:
#predict_submission = model.predict(X_sub)
#predict_submission

In [ ]:
#predict_submission = gb_model.predict(X_sub)
#predict_submission

In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_v{VERSION}.csv', index=False)
sample_submission.head(10)

# What's next?
Или что еще можно сделать, чтоб улучшить результат:

* Спарсить свежие данные 
* Посмотреть, что можно извлечь из признаков или как еще можно обработать признаки
* Сгенерировать новые признаки
* Попробовать подобрать параметры модели
* Попробовать другие алгоритмы и библиотеки ML
* Сделать Ансамбль моделей, Blending, Stacking